In [1]:
import pickle
import pandas as pd

In [ ]:
text1=open('dialogs.txt','r').read().splitlines()
text2=pd.read_csv('Conversation.csv')

In [3]:
text2['text'] = text2.question +" " + text2.answer

In [4]:
text2=text2['text'].to_list()

In [5]:
text=text1 + text2

In [6]:
len(text)

7455

In [7]:
class Tokenizer:
    def __init__(self,vocab_size=500):
        self.vocab_size=vocab_size
        self.general_vocab=256
        self.sos="<|SOS|>"
        self.eos="<|EOS|>"
        self.ukn="<|UKN|>"
        self.enc_eos=vocab_size-1
        self.enc_sos=vocab_size-2
        self.enc_ukn=vocab_size-3
        self.merges={}
        self.vocab={}
    
    def _to_token(self,text)->list:
        tokens=[]
        for sen in text:
            toks=list(map(int,sen.encode('utf-8')))
            toks=[self.enc_sos] + toks + [self.enc_eos]
            for t in toks:
                tokens.append(t)
        return tokens
    def _get_stats(self,ids):
        frequency={}
        for p in zip(ids,ids[1:]):
            frequency[p] = frequency.get(p,0) + 1
        return frequency

    def _merge(self,ids,pair,idx):
        new_ids=[]
        i=0
        while i<len(ids):
            if i<len(ids)-1 and ids[i]==pair[0] and ids[i+1] == pair[1]:
                new_ids.append(idx)
                i+=2
            else:
                new_ids.append(ids[i])
                i+=1
        return new_ids

    def train(self,text):
        tokens=self._to_token(text)
        for i in range(self.vocab_size - self.general_vocab-1):
            idx=self.general_vocab + i
            stats=self._get_stats(tokens)
            pair=max(stats,key=stats.get)
            tokens=self._merge(tokens,pair,idx)
            self.merges[pair] = idx
        self.vocab={idx : bytes([idx]) for idx in range(self.general_vocab)}
        self.vocab[self.enc_sos] = bytes(self.sos.encode('utf-8'))
        self.vocab[self.enc_eos] = bytes(self.eos.encode('utf-8'))
        self.vocab[self.enc_ukn] = bytes(self.ukn.encode('utf-8'))
        for (p0,p1),enc in (self.merges.items()):
            self.vocab[enc] = self.vocab[p0] + self.vocab[p1]
    def decode(self,tokens):
        result = []
        for t in tokens:
            if t == self.enc_sos:
                result.append(self.sos)
            elif t == self.enc_eos:
                result.append(self.eos)
            elif t == self.enc_ukn:
                result.append(self.ukn)
            elif t in self.vocab:
                try:
                    result.append(self.vocab[t].decode('utf-8'))
                except Exception:
                    result.append(self.ukn)
            else:
                result.append(self.ukn)
        return ''.join(result)
    def encode(self,text):
        tokens=list(map(int,text.encode('utf-8')))
        tokens=[self.enc_sos] + tokens + [self.enc_eos]
        while len(tokens)>1:
            stats=self._get_stats(tokens)
            pair=min(stats,key=lambda p:self.merges.get(p,float('inf')))
            if pair not in self.merges:
                break
            idx=self.merges.get(pair,self.enc_ukn)
            tokens=self._merge(tokens,pair,idx)
        tokens=[t if t in self.vocab else self.enc_ukn for t in tokens]
        return tokens

In [8]:
tokenizer=Tokenizer(4000)
tokenizer.train(text)

In [19]:

with open('tokenizer.pkl','wb') as f:
    pickle.dump(tokenizer,f)

In [20]:
with open('tokenizer.pkl','rb') as f:
    model=pickle.load(f)

In [24]:
test="""The scientist studied the effects of xenotransplantation."""

In [25]:
tokens=model.encode(test)

In [26]:
for tok in tokens:
    print(f"{tok} -> {model.decode([tok])}")

3998 -> <|SOS|>
84 -> T
2977 -> he s
553 -> ci
283 -> en
350 -> ti
465 -> st 
895 -> stu
305 -> di
1342 -> ed the 
101 -> e
1311 -> ffe
99 -> c
470 -> ts 
328 -> of 
120 -> x
283 -> en
111 -> o
1361 -> tr
270 -> an
115 -> s
382 -> pl
819 -> ant
907 -> ation
46 -> .
3999 -> <|EOS|>
